In [1]:
# import libraries
import pandas as pd
import numpy as np

import xml.etree.ElementTree as ET


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

# load the data set
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/reviews.csv', names = ['top_id', 'date_modified', 'aspect_review'], header=None)

print(df.shape)
df.head()

(606591, 3)


top_id            date_modified  \
0  115447  2024-05-20 15:04:36.770   
1  115447  2024-06-01 14:11:14.807   
2  115447  2024-06-08 17:31:13.090   
3  115447  2024-06-15 14:39:17.800   
4    2518  2024-05-20 10:48:33.227   

                                       aspect_review  
0  <user_reviews><user_review date="a year ago"><...  
1  <user_reviews><user_review date="a year ago"><...  
2  <user_reviews><user_review date="a year ago"><...  
3  <user_reviews><user_review date="a year ago"><...  
4  <user_reviews><user_review date="5 months ago"...

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606591 entries, 0 to 606590
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   top_id         606591 non-null  int64 
 1   date_modified  606591 non-null  object
 2   aspect_review  606591 non-null  object
dtypes: int64(1), object(2)
memory usage: 13.9+ MB


In [11]:
#Extract XML data
def extract_aspects(xml_data):
    # Handle the case where xml_data is empty or invalid
    if not xml_data or not xml_data.strip():
        return []  # Return an empty list if no data is available

    try:
        root = ET.fromstring(xml_data)
        reviews = []

        # Iterate through each review in the XML
        for review in root.findall('user_review'):
            review_data = {
                'date': review.get('date'),
            }

            # Check if 'text' tag exists before accessing 'text' attribute
            text_element = review.find('text')
            if text_element is not None:
                review_data['text'] = text_element.text
            else:
                review_data['text'] = ''  # or handle the missing text as needed

            reviews.append(review_data)
        return reviews

    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return []  # Return an empty list on parsing error

# Apply the extraction function to each row
df['extracted_reviews'] = df['aspect_review'].apply(extract_aspects)

# Investigate potential issues
problematic_rows = df[df['extracted_reviews'].apply(lambda x: len(x) == 0)]
print("Number of rows with parsing issues:", len(problematic_rows))

# Inspect the first few problematic rows
print(problematic_rows.head())


Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: unclosed token: line 1, column 15942
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Error parsing XML: no element found: line 1, column 16000
Number of rows with parsing issues: 11
        top_id            date_modified  \
111672    3595  2024-05-20 13:48:18.313   
212927    3595  2024-05-25 17:47:20.770   
223116      18  2024-05-25 12:42:49.607   
223141  278914  2024-05-25 13:25:50.327   
310717    3595  2024-06-01 13:13:06.243   

                                            aspect_review extracte

In [12]:
exploded_df = df.explode('extracted_reviews').reset_index(drop=True)
exploded_df = pd.concat([exploded_df.drop(['extracted_reviews'], axis=1), exploded_df['extracted_reviews'].apply(pd.Series)], axis=1)


In [13]:
final_df = exploded_df
print(final_df.shape)
final_df.head()

(4808273, 6)


top_id            date_modified  \
0  115447  2024-05-20 15:04:36.770   
1  115447  2024-05-20 15:04:36.770   
2  115447  2024-05-20 15:04:36.770   
3  115447  2024-05-20 15:04:36.770   
4  115447  2024-05-20 15:04:36.770   

                                       aspect_review          date  \
0  <user_reviews><user_review date="a year ago"><...    a year ago   
1  <user_reviews><user_review date="a year ago"><...  3 months ago   
2  <user_reviews><user_review date="a year ago"><...   6 years ago   
3  <user_reviews><user_review date="a year ago"><...   5 years ago   
4  <user_reviews><user_review date="a year ago"><...   4 years ago   

                                                text   0  
0  Popped in here randomly on a Sunday to get a b... NaN  
1  Everything was delicious! Even my husband, who... NaN  
2  The food is fresh, modern, and flavoursome. Th... NaN  
3  It’s a nice place to have a week end brunch wi... NaN  
4  Nice Scottish breakfast but service a little s... NaN

In [16]:
#drop unused columns
final_df = final_df[['top_id', 'date_modified', 'text']] # Use a list of column names within square brackets
print(final_df.shape)
final_df.head()

(4808273, 3)


top_id            date_modified  \
0  115447  2024-05-20 15:04:36.770   
1  115447  2024-05-20 15:04:36.770   
2  115447  2024-05-20 15:04:36.770   
3  115447  2024-05-20 15:04:36.770   
4  115447  2024-05-20 15:04:36.770   

                                                text  
0  Popped in here randomly on a Sunday to get a b...  
1  Everything was delicious! Even my husband, who...  
2  The food is fresh, modern, and flavoursome. Th...  
3  It’s a nice place to have a week end brunch wi...  
4  Nice Scottish breakfast but service a little s...

In [18]:
final_df.head(10)


top_id            date_modified  \
0  115447  2024-05-20 15:04:36.770   
1  115447  2024-05-20 15:04:36.770   
2  115447  2024-05-20 15:04:36.770   
3  115447  2024-05-20 15:04:36.770   
4  115447  2024-05-20 15:04:36.770   
5  115447  2024-05-20 15:04:36.770   
6  115447  2024-05-20 15:04:36.770   
7  115447  2024-05-20 15:04:36.770   
8  115447  2024-06-01 14:11:14.807   
9  115447  2024-06-01 14:11:14.807   

                                                text  
0  Popped in here randomly on a Sunday to get a b...  
1  Everything was delicious! Even my husband, who...  
2  The food is fresh, modern, and flavoursome. Th...  
3  It’s a nice place to have a week end brunch wi...  
4  Nice Scottish breakfast but service a little s...  
5  Food was nice, service was ok, we ordered the ...  
6  A little bit of the beaten paths is the Treacl...  
7  This place was okay, however I felt it was ove...  
8  Popped in here randomly on a Sunday to get a b...  
9  Everything was delicious! Even my husband, who...

In [25]:
# Remove duplicate rows
final_df = final_df.drop_duplicates(subset=['top_id', 'text'])

In [26]:
# Check if there are any duplicates
has_duplicates = final_df.duplicated(subset=['top_id', 'text']).any()

print(f"Are there any duplicates? {has_duplicates}")

Are there any duplicates? False


In [27]:
print(final_df.shape)
final_df.head(10)

(943102, 3)


top_id            date_modified  \
0   115447  2024-05-20 15:04:36.770   
1   115447  2024-05-20 15:04:36.770   
2   115447  2024-05-20 15:04:36.770   
3   115447  2024-05-20 15:04:36.770   
4   115447  2024-05-20 15:04:36.770   
5   115447  2024-05-20 15:04:36.770   
6   115447  2024-05-20 15:04:36.770   
7   115447  2024-05-20 15:04:36.770   
32    2518  2024-05-20 10:48:33.227   
33    2518  2024-05-20 10:48:33.227   

                                                 text  
0   Popped in here randomly on a Sunday to get a b...  
1   Everything was delicious! Even my husband, who...  
2   The food is fresh, modern, and flavoursome. Th...  
3   It’s a nice place to have a week end brunch wi...  
4   Nice Scottish breakfast but service a little s...  
5   Food was nice, service was ok, we ordered the ...  
6   A little bit of the beaten paths is the Treacl...  
7   This place was okay, however I felt it was ove...  
32  So glad that this lovely pub has reopened. Has...  
33  Wow! What a wonderful evening being able to di...

In [28]:
#save as new csv file for further use
final_df.to_csv('Extracted_reviews.csv', index=False)